In [1]:
from watsonx import WatsonxAI
import warnings
from colorama import Fore, Back

warnings.filterwarnings("ignore")

from dotenv import load_dotenv
from os import getenv

load_dotenv()
BRAVE_API_KEY = getenv("BRAVE_API_KEY", None)


proxy = "proxy.us.ibm.com:8080"

wx = WatsonxAI()
wx.connect()

In [2]:
def gen_naive_answer(question):
    """
    get the AI answer the question directly.
    """

    prompt = f"""<|user|>
please answer the question.
<<SYS>>
question: `{question}`
<</SYS>>
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

answer = gen_naive_answer("how many r in strawberry?")
print(answer)

The word "strawberry" contains one 'r'.


# 1) reflection

In [ ]:
def gen_review(draft):
    prompt = f"""<|user|>
please review the draft provided.
-check if any illogical content.
-check if any unclear content.
-give contructive suggestion.
draft: `{draft}`
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

def gen_revise(content,review):
    prompt = f"""<|user|>
please revise the content base on the review provided.
-well adopt the suggestion provided.
-generate the revised content only.
content: `{content}`
review: `{review}`
<|assistant|>
revised version:
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

requirement = "plan a trip to jakarta for 3 days"

draft = gen_naive_answer(requirement)
print("draft:")
print(Fore.BLUE, draft)

review = gen_review(draft)
print("review:")
print(Fore.RED, review)

revise = gen_revise(draft,review)
print("revise:")
print(Fore.YELLOW, revise)

# Pattern 1: Planning

In [ ]:
def gen_thought(requirement):
    """
    get the AI not jump to conclusion or action, try approach a problem with thought first.
    """

    prompt = f"""<|system|>
you are a chatbot that answer general question.
-understand the requirement in detail.
-list out what knowlege you need to answer the requirement.
-take the simplest solution.
-classify what domain this requirement about.
-think about where will the right source of the answer.
-think logically.
-think step by step.
-you trust the result of the tools more than your memory.
-you trust the knowledge provide more than your memory.
-leverage tool if possible.
-dont guess.
-dont overcomplicate.
-declare the assumption you made.
-NEVER generate Final Result
<|user|>
please generate your thought on the question provided.
<<SYS>>
requirement: `{requirement}`
<</SYS>>
<|assistant|>
thought:
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

requirement = "how many r in strawberry?"
thought = gen_thought(requirement)
print(thought)

In [ ]:
def gen_plan(requirement):
    """
    get the API to break down the big task to smaller sub-tasks. that easier to execute
    """
    
    prompt = f"""<|system|>
-dont overcomplicate.
-think logically.
-think step by step.
-dont guess.
-you trust the result of the tools more than your memory.
-you trust the knowledge provide more than your memory.
-you trust the status provided. no need to check on it.
<|user|>
please generate a concise and solid plan to satisfy the requirement provided.
-understand the requirement in detail.
-understand the requirement with common sense.
-understand the status provided.
-list out what knowlege you need to satisfy the requirement.
-leverage tool if possible.
-please breakdown complex task to simple tasks.
<<SYS>>
requirement: `{requirement}`
<</SYS>>
-DONT generate result of the steps.
-DONT generate status.
-**DONT** generate **Final Result:**
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

def gen_next_step(plan,status):
    """
    get the API to break down the big task to smaller sub-tasks. that easier to execute
    """
    
    prompt = f"""<|system|>
<|user|>
please check the status against the plan provided.
-understand the plan provided in detail.
-understand the status provided in detail.
-generate next step base on status if has next step.
-print 'complete' if no next step.
<<SYS>>
plan: `{plan}`
status: `{status}`
output format: {{
    next_step:,
    complete: yes/no,
}}
<</SYS>>
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

plan = gen_plan("goto Jakarta from Singapore")
print("plan:")
print(plan)

next_step = gen_next_step(plan,"I have the visa and passport")
print("next step:")
print(next_step)

next_step = gen_next_step(plan,"already in Jakarta")
print("next step:")
print(next_step)

# Pattern 3: Use Tool

In [ ]:
import inspect
import ast
import json

def get_function_signature_and_docstring(func):
    source_lines = inspect.getsourcelines(func)[0]
    source = ''.join(source_lines)

    # Parse the function definition using AST (Abstract Syntax Trees)
    parsed_func = ast.parse(source)

    # Extract function signature and docstring
    func_signature = ''
    docstring = ''
    return_type = ''
    for node in parsed_func.body:
        if isinstance(node, ast.FunctionDef):
            func_signature = source_lines[node.lineno - 1].strip()
            docstring = ast.get_docstring(node) or ''
            # Get return type annotation
            return_type = node.returns.id if node.returns else 'None'
            break

    return func_signature, docstring, return_type

def format_function_info(func):
    func_signature, docstring, return_type = get_function_signature_and_docstring(func)
    function_name = func.__name__

    # Extract parameters from a JSON schema-like structure
    parameters = {
        "type": "object",
        "properties": {}
    }

    # Get the function's parameters and their annotations
    func_params = inspect.signature(func).parameters
    for param_name, param in func_params.items():
        param_type = param.annotation
        param_description = ""  # You can add a description here if needed
        parameters["properties"][param_name] = {
            "type": str(param_type),
            "description": param_description
        }

    # Set the required parameters based on function annotations
    required_params = [param_name for param_name, param in func_params.items() if param.default == inspect.Parameter.empty]
    parameters["required"] = required_params

    return {
        "name": function_name,
        "parameters": parameters,
        "description": docstring,
        "return_type": return_type
    }

def get_tools_prompt(tools):
    prompt = "["
    # Get the source code of the function
    for tool in tools:
        info = format_function_info(tool)
        prompt = prompt + f"{info}\n"
    prompt = prompt + "]"
    return prompt

def pick_tool(job,thought,tools):
    """
    pick a tool from toolbox, that best match the need for requirement.
    """

    prompt = f"""<|system|>
You are a helpful assistant with access to the following function calls. 
- Your task is to produce a sequence of function calls necessary to generate response to the user utterance. 
- well understand the thought to articulate the task.
- well understand the description and parameters of tools.
= select appropriate parameter for the tool.
- only pick the function required and provided in tools.
- show "missing tool" if no appropriate function provided.
- DONT explain.
output format: [{{"name": "function", "arguments": {{"arg": "value"}}}}]
<<TOOLS>>
{get_tools_prompt(tools)}
<</TOOLS>>
<|user|>
thought: `{thought}`
job: `{job}`
<|assistant|>
"""
    # print(prompt)
    # print(">>"+step)
    tool = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return tool

def use_tool(func_json_str):
    """
    you already hv the tool, let run it.
    """
    try:
        # Parse the JSON string into Python objects
        function_calls = json.loads(func_json_str)
        
        # Dictionary to store results
        results = {}
        
        # Iterate over each function call in the list
        for call in function_calls:
            func_name = call.get("name")
            args = call.get("arguments", {})
            print(f"calling {func_name}({args})...")
            
            # Check if the function exists in the global namespace
            if func_name in globals() and callable(globals()[func_name]):
                try:
                    # Retrieve the function object
                    func = globals()[func_name]
                    
                    # Call the function with the provided arguments
                    result = func(**args)
                    
                    # Store the result
                    results[f"{func_name}({args})"] = result
                except Exception as e:
                    # Handle exceptions raised by the function
                    results[f"{func_name}({args})"] = f"Error executing {func_name}: {str(e)}"
            else:
                # Function not found or not callable
                results[f"{func_name}({args})"] = f"Function '{func_name}' not found or is not callable."
        
        return results
    
    except json.JSONDecodeError as jde:
        return {"JSONDecodeError": f"Invalid JSON input: {str(jde)}"}
    except Exception as ex:
        return {"Error": f"An unexpected error occurred: {str(ex)}"}
    
def gen_summary_from_knowledge(question,knowledge):
    prompt = f"""<|user|>
help answer the question provided base on the knowledge provided.
-focus on what you found instead of not found.
-if no answer provide, just say you dont know.
<<SYS>>
question = `{question}`
knowledge = `{knowledge}`
<</SYS>>
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT,8000)
    return answer

In [ ]:
import requests
import wikipediaapi
import requests
import yfinance as yf
import pytz
from datetime import datetime

from dotenv import load_dotenv
from os import getenv

load_dotenv()
PROXY = getenv("PROXY", None)

def query_wikipedia(topic)->str:
    """
    only use it for ONE keyword/topic. 
    NEVER applicable to long query.
    get detail definition and information of particular topic
    only ONE keyword/topic will be support. please use appropriate one.
    only support very specific short keyword or topic.

    Parameters:
    topic (str): The ONE keyword/topic for which the information is required.

    Returns:
    str: A dictionary containing the title, summary, and URL of the Wikipedia page for the given topic. If the page is not found, it returns "Page not found."
    """
    wiki = wikipediaapi.Wikipedia('watsonx',proxies={'http': PROXY,'https':PROXY})
    page = wiki.page(topic)

    if page.exists():
        return {
            "title": page.title,
            "summary": page.summary,
            "url": page.fullurl
        }
    else:
        return "Page not found."

def get_stock_data(ticker, period='5d', interval='1d')->str:
    """
    get stock price. it is for stock data only.
    the input is ticker (should be a stock code)
    dont put other thing to ticker.
    """
    session = requests.Session()
    session.proxies = {
        'http': PROXY,
        'https': PROXY
    }

    stock = yf.Ticker(ticker, session=session)
    data = stock.history(period=period, interval=interval)
    return data

def get_current_time(location)->str:
    """
    Get the current time by location.

    Args:
        location (str) need to be in format of 'Europe/London'

    Returns:
        str: The current time in the specified location.

    Example:
        get_current_time('Europe/London')
    """
    try:
        location_time = datetime.now(pytz.timezone(location))
        return location_time.strftime("%Y-%m-%d %H:%M:%S")
    except Exception as e:
        return f"Error: {str(e)}"

tools = [query_wikipedia,get_stock_data,get_current_time]

In [7]:
def run_agentic_from_tool(question,tools,debug=False):
    # print(get_tools_prompt(tools))
    # thought = gen_thought(question)
    action = pick_tool(question,"",tools)
    if debug:
        print(f"action:{action}")
    answer = use_tool(action)
    if debug:
        print(f"tool result:{answer}")
    summary = gen_summary_from_knowledge(question,answer)
    return summary

In [21]:
answer = gen_naive_answer("how deep is Mariana Trench")
print("naive:")
print(answer)

print("use tool:")
answer = run_agentic_from_tool("how deep is Mariana Trench",tools)
print(answer)

naive:
The Mariana Trench, located in the western Pacific Ocean, is approximately 36,070 feet (10,994 meters) deep at its deepest point, known as the Challenger Deep. This makes it the deepest part of the world's oceans.
use tool:
calling query_wikipedia({'topic': 'Mariana Trench'})...
The Mariana Trench, located in the western Pacific Ocean, is the deepest oceanic trench on Earth. Its maximum known depth is 10,984 ± 25 meters (36,037 ± 82 feet; 6,006 ± 14 fathoms; 6.825 ± 0.016 miles). This depth is more than 2 kilometers (1.2 miles) farther from sea level than the peak of Mount Everest. The pressure at the bottom of the trench is 1,086 bar (15,750 psi), and the temperature ranges from 1 to 4 degrees Celsius (34 to 39 degrees Fahrenheit). One-celled organisms called monothalamea have been found at a record depth of 10.6 kilometers (35,000 feet; 6.6 miles) below the sea surface.


In [24]:
answer = gen_naive_answer("analysis the latest stock price of IBM")
print("naive:")
print(answer)

print("use tool:")
answer = run_agentic_from_tool("analysis the latest stock price of IBM",tools)
print(answer)

naive:
As of my last update, I don't have real-time data or the ability to browse the internet. However, I can guide you on how to find the latest stock price of IBM. You can check financial websites like Yahoo Finance, Google Finance, or Bloomberg for the most recent stock prices. Please ensure you're using a reliable source and that the information is up-to-date.
use tool:
calling get_stock_data({'ticker': 'IBM'})...
Based on the provided knowledge, the latest stock price of IBM as of November 18, 2024, was $208.089996. This is the closing price for that day. The volume of shares traded on that day was 2658508. There were no dividends or stock splits reported for this period.


In [23]:
answer = gen_naive_answer("when it is now in Sydney")
print("naive:")
print(answer)

print("use tool:")
answer = run_agentic_from_tool("when it is now in Sydney",tools)
print(answer)

naive:
The current time in Sydney, Australia is [insert current time here]. Please note that Sydney observes Daylight Saving Time from early October to early April, during which the time zone is UTC+10:30. The rest of the year, it follows UTC+11:00.
use tool:
calling get_current_time({'location': 'Australia/Sydney'})...
The current time in Sydney is 13:36:25 on November 19, 2024.


In [ ]:
wx.connect()

# answer = query_wikipedia("olympics")
# # print(answer)

# answer = gen_naive_answer( "who win the 47th president of USA")
# print(answer)

# answer = gen_answer_from_knowledge("who win the 47th president of USA",tools,debug=True)
# print(answer)

answer = run_agentic_from_tool("who win the 47th president of USA",tools)
print(answer)

In [ ]:
answer = gen_naive_answer( "when it is now in hong kong")
print(answer)

answer = gen_answer_from_knowledge("how many hour different of hong kong to SFO",tools)
print(answer)

answer = run_agentic_from_tool("how many hour different of hong kong to SFO",tools)
print(answer)

In [ ]:
answer = gen_naive_answer("compare UK and france")
print(answer)

answer = gen_answer_from_knowledge("compare UK and france",tools)
print(answer)

answer = run_agentic_from_tool("compare UK and france",tools)
print(answer)

In [ ]:
answer = gen_naive_answer("compare IBM and AMAZON stock price today")
print(answer)

answer = gen_answer_from_knowledge("compare IBM and AMAZON stock price today",tools)
print(answer)

answer = run_agentic_from_tool("compare IBM and AMAZON stock price today",tools)
print(answer)

In [ ]:
answer = run_agentic_from_tool("who is competitor of vllm",tools)
print(answer)

In [ ]:
answer = run_agentic_from_tool("will it be typhoon tmr in hong kong",tools)
print(answer)

In [ ]:
answer = gen_answer_from_knowledge("stock price of IBM compare to MSFT",tools)
print(answer)

answer = run_agentic_from_tool("stock price of IBM compare to MSFT",tools)
print(answer)

In [ ]:
answer = gen_answer_from_knowledge("compare Mickey Mouse and Indiana Jones")
print(answer)

answer = gen_answer_from_knowledge("compare Trump and Biden")
print(answer)

answer = gen_answer_from_knowledge("tell me about star war")
print(answer)

answer = gen_answer_from_knowledge("tell me about beijing and paris on olympics")
print(answer)

answer = gen_answer_from_knowledge("tell me about TVB")
print(answer)

answer = gen_answer_from_knowledge("stock price of IBM")
print(answer)

answer = gen_answer_from_knowledge("what is official language of singapore")
print(answer)

# 4) solve problem by coding

In [ ]:
import io
import sys
import traceback

def gen_code(requirement,code,plan,error,tools,debug=False):
    prompt = f"""<|user|>
please generate python code that execute the plan to fulfill the requirement.
-understand the requriement in detail.
-understand the tool provided.
-ensure the python generated is executable.
-clean up if any unneccessary characters.
-review the problem in the code and the error if provided.
-prevent the error from the code if provided.
-involve the tool provided directly without define it.
-understand the plan in detail. dont miss the steps.
-**DONT** use SaaS service which need API KEY.
-**DONT** catch error.
-raise error when hit problem.
-**DONT** request input.
-**DONT** throw SystemExit
-**DONT** generate main().
-**DONT** generate explanation.
-ensure print the answer to output explicitely.
-generate the code directly.
-generate the code only.
<<SYS>>
requirement: `{requirement}`
code with problem: `{code}`
plan: `{plan}`
error: `{error}`
-involve the tool provided directly without define it.
tools: `{get_tools_prompt(tools)}`
<</SYS>>
-dont generate explanation.
-dont generate note.
-generate the code directly only.
<|assistant|>
```python"""
    if debug:
        print(prompt)
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT).replace("```","")
    return answer

def create_local_namespace(tools):
    return {func.__name__: func for func in tools}

def run_code(module_import, function_call, tools, max_traceback_lines=5):
    """
    try to run the generated code to get the answer.
    """
    # Create StringIO objects to capture standard output and error output
    output_stream = io.StringIO()
    error_stream = io.StringIO()

    # Redirect stdout and stderr
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    sys.stdout = output_stream
    sys.stderr = error_stream

    output = None
    error = None

    # Using exec to import the module and evaluate the function call
    try:
        local_namespace = create_local_namespace(tools)
        exec(module_import, local_namespace, local_namespace)  # Dynamically import the module
        # print(f"|{function_call}|")
        exec(function_call, local_namespace, local_namespace)  # Evaluate the function call
    except ImportError as e:
        error = f"Import Error: {str(e)}"
    except Exception as e:
        # Capture the traceback for more detailed error information
        full_traceback = traceback.format_exc().strip()
        # Limit the traceback to the last 'max_traceback_lines' lines
        limited_traceback = "\n".join(full_traceback.splitlines()[-max_traceback_lines:])
        error = f"Unexpected Error: {str(e)}\n{limited_traceback}"
    finally:
        # Reset stdout to its original state
        sys.stdout = old_stdout
        sys.stderr = old_stderr

    # Get the output from the output_stream
    output = output_stream.getvalue().strip()
    error_output = error_stream.getvalue().strip()

    # Check the output or error
    if error:
        return f"Output: {output}", f"Error: {error}"
    elif error_output:
        return f"Output: {output}", f"Error Output: {error_output}"
    else:
        return f"Output: {output}", None
    
def gen_eval(question,stdout):
    prompt = f"""<|user|>classify if stdout include error.
-show sucess or fail only, but nothing else.
-show success if include no error. and end the generation.
-show fail if include error. and end the generation.
<<SYS>>
question: `{question}`
stdout: `{stdout}`
<</SYS>>
<|assistant|>success or fail:"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT).replace("```","")
    return answer
    
def gen_code_summary(requirement,code,output):
    prompt= f"""<|user|>
please generate a complete sentence summary base on the answer provided only.
-dont use your common sense.
-declare the assumption you made.
-only base on the requirement and answer provided.
-the output comes from the code execution.
-dont guess.
<<SYS>>
requirement: `{requirement}`
code: `{code}`
answer: `{output}`
-dont mention about code.
-use simple english.
<</SYS>>
<|assistant|>answer summary:"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT).replace("```","").replace("<|user|>","")
    return answer

def run_agentic_with_coding(requirement, max_turns=10, debug=False):
    """
    gen thought or plan first.
    for each turns:
        gen code base on plan until output is error free
    gen summary
    """
    thought = gen_thought(requirement)
    if debug:
        print(f"thought: {thought}")

    plan = thought

    code = ""
    error = ""
    # Loop until the review output is empty
    for turn in range(max_turns):
        print(f"turn #{turn+1} ---")
        # output = exec_with_output(code)
        code = gen_code(requirement, code , plan, error, tools)
        if debug:
            print(f"code: {code}")
        output, error = run_code("",code, tools)
        if debug:
            print(f"output: {output}")
        
        if error is None:
            break
        else:
            if debug:
                print(f"error {error}")
        # review = eval_output(requirement, output)
        # if debug:
        #     print(f"#4: {review}")
        
        # if "success" in review:  # Exit if review output is empty
        #     # print(f"#final result: {output}")
        #     break

    summary = gen_code_summary(requirement, code, output)
    return summary

In [ ]:
output = run_agentic_with_coding("tell me is it only 100 prime number with in 10000?")
print(output)

output = run_agentic_with_coding("tell me is it only 100 prime number with in 1000?")
print(output)

output = run_agentic_with_coding("tell me all prime number with in 1000?")
print(output)

output = run_agentic_with_coding("tell me all prime number with in 1000? and it need to be ends with 7")
print(output)

output = run_agentic_with_coding("how many prime number with in 10000")
print(output)

output = run_agentic_with_coding("how many R in straberry")
print(output)

output = run_agentic_with_coding("how many r in straberry")
print(output)

output = run_agentic_with_coding("how many r/R in straberry")
print(output)

output = run_agentic_with_coding("how many 'a' in canada")
print(output)

output = run_agentic_with_coding("how many A in 'canada', dont check case")
print(output)

output = run_agentic_with_coding("how many a or A in the word canada")
print(output)

output = run_agentic_with_coding("in math, compare 9.11 with 9.8")
print(output)

output = run_agentic_with_coding("9.1 9.11 and 9.21, 9.8, which is bigger? and which is smaller")
print(output)

output = run_agentic_with_coding("compare 9.11 with 9.8")
print(output)

output = run_agentic_with_coding("which of 9.11 9.8 is bigger")
print(output)

output = run_agentic_with_coding("compare 9.11 with 9.A")
print(output)

output = run_agentic_with_coding("tell me Korea time")
print(output)

output = run_agentic_with_coding("tell me how is the different of Korea time and Hong Kong time")
print(output)

output = run_agentic_with_coding("tell me Hong Kong time")
print(output)

output = run_agentic_with_coding("search Hong Kong on wiki",debug=True)
print(output)

output = run_agentic_with_coding("tell me about Paris",debug=True)
print(output)